In [ ]:
!pip install pandas

In [1]:
import datetime
from datetime import datetime as dt
import pandas as pd

from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters
from google.cloud.bigtable.row_set import RowSet 

In [2]:
INSTANCE_ID = 'big-table-db2'
TABLE_GENERATION_ID = 'plant_generation'
TABLE_WEATHER_ID = 'plant_weather'

In [3]:
client = bigtable.Client.from_service_account_json('../../lofty-psyche-328203-949244c34274.json', admin = True)

In [4]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


In [5]:
print('Creating the {} table'.format(TABLE_GENERATION_ID))
table_generation = instance.table(TABLE_GENERATION_ID)

if not table_generation.exists():
    table_generation.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_GENERATION_ID))

print('Creating the {} table'.format(TABLE_WEATHER_ID))
table_weather = instance.table(TABLE_WEATHER_ID)

if not table_weather.exists():
    table_weather.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_WEATHER_ID))

Creating the plant_generation table
ERROR: Table plant_generation already exists
Creating the plant_weather table
ERROR: Table plant_weather already exists


In [ ]:
print('Creating family columns')

plant_cfId = 'plant'
plant_cf = table_weather.column_family(plant_cfId)
plant_cf.create()



In [ ]:
telemetry_cfId = 'telemetry'
telemetry_cf = table_weather.column_family(telemetry_cfId)
telemetry_cf.create()

In [ ]:
generation_cfId = 'generation'
items_cf = table_generation.column_family(generation_cfId)
items_cf.create()

In [9]:
df_weather_plant2 = pd.read_csv('../data/Plant_2_Weather_Sensor_Data.csv')
df_weather_plant1 = pd.read_csv('../data/Plant_1_Weather_Sensor_Data.csv')
df_generation_plant2 = pd.read_csv('../data/Plant_2_Generation_Data.csv')
df_generation_plant1 = pd.read_csv('../data/Plant_1_Generation_Data.csv')

In [ ]:
print('Writing to the plant 2 weather table')
rows = []
for line in range(len(df_weather_plant2 )):
    row_key = 'plant#{}#{}'.format(df_weather_plant2 ['PLANT_ID'][line], df_weather_plant2 ['DATE_TIME'][line]).encode()
    date_time = datetime.datetime.strptime(df_weather_plant2 ['DATE_TIME'][line], '%Y-%m-%d %H:%M:%S')
    row = table_weather.direct_row(row_key)

    row.set_cell(plant_cfId, 'source_key'.encode(), df_weather_plant2 ['SOURCE_KEY'][line], timestamp = date_time)

    row.set_cell(telemetry_cfId, 'ambient_temp'.encode(), str(df_weather_plant2 ['AMBIENT_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'module_temp'.encode(), str(df_weather_plant2 ['MODULE_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'irradiation'.encode(), str(df_weather_plant2 ['IRRADIATION'][line]), timestamp = date_time)

    rows.append(row)

table_weather.mutate_rows(rows)

In [ ]:
print('Writing to the plant 1 weather table')
rows = []
for line in range(len(df_weather_plant1 )):
    row_key = 'plant#{}#{}'.format(df_weather_plant1 ['PLANT_ID'][line], df_weather_plant1 ['DATE_TIME'][line]).encode()
    date_time = datetime.datetime.strptime(df_weather_plant1 ['DATE_TIME'][line], '%Y-%m-%d %H:%M:%S')
    row = table_weather.direct_row(row_key)

    row.set_cell(plant_cfId, 'source_key'.encode(), df_weather_plant1 ['SOURCE_KEY'][line], timestamp = date_time)

    row.set_cell(telemetry_cfId, 'ambient_temp'.encode(), str(df_weather_plant1 ['AMBIENT_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'module_temp'.encode(), str(df_weather_plant1 ['MODULE_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'irradiation'.encode(), str(df_weather_plant1 ['IRRADIATION'][line]), timestamp = date_time)

    rows.append(row)

table_weather.mutate_rows(rows)

In [12]:
range(len(df_generation_plant1))

range(0, 68778)

In [16]:
df = df_generation_plant1.iloc[0:3000]
df

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,0.000000,6259559.000
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.000000,0.000000,0.000000,6183645.000
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.000000,0.000000,0.000000,6987759.000
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.000000,0.000000,0.000000,7602960.000
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.000000,0.000000,0.000000,7158964.000
...,...,...,...,...,...,...,...
2995,16-05-2020 13:45,4135001,ZoEaEvLYb1n2sOq,6888.125000,674.525000,4626.500000,7108986.500
2996,16-05-2020 13:45,4135001,adLQvlD726eNBSB,6833.285714,669.228571,4902.428571,6282640.429
2997,16-05-2020 13:45,4135001,bvBOhCH3iADSZry,6453.375000,632.075000,4487.750000,6326997.750
2998,16-05-2020 13:45,4135001,iCRJl6heRkivqQ3,6954.250000,681.012500,4640.125000,7189050.125


In [13]:
print('Writing to the generation table')
rows = []
acumulador = 0
for line in range(len(df_generation_plant1)):
    df = df_generation_plant1.iloc[acumulador:acumulador+3000]
    for row in range(len(df)):
        row_key = 'plant#{}#{}'.format(df['PLANT_ID'][line], df['DATE_TIME'][line]).encode()
        date_time = datetime.datetime.strptime(df['DATE_TIME'][line], '%d-%m-%Y %H:%M')
        row = table_generation.direct_row(row_key)

        row.set_cell(plant_cfId, 'source_key'.encode(), df['SOURCE_KEY'][line], timestamp = date_time)

        row.set_cell(generation_cfId, 'dc_power'.encode(), str(df['DC_POWER'][line]), timestamp = date_time)
        row.set_cell(generation_cfId, 'ac_power'.encode(), str(df['AC_POWER'][line]), timestamp = date_time)

        row.set_cell(generation_cfId, 'daily_yield'.encode(), str(df['DAILY_YIELD'][line]), timestamp = date_time)
        row.set_cell(generation_cfId, 'total_yield'.encode(), str(df['TOTAL_YIELD'][line]), timestamp = date_time)

        rows.append(row)
    acumulador += 3000
    table_generation.mutate_rows(rows)

Writing to the generation table


KeyError: 1

In [11]:
print('¿Cuáles fueron las diferentes medidas de temperatura ambiente para la planta 2 con fecha del 1 de junio desde las 6 de la mañana al mediodía?')

row_set = RowSet()
row_set.add_row_range_from_keys(
    start_key = b'plant#4136001#2020-06-01 06:00:00',
    end_key = b'plant#4136001#2020-06-01 12:00:00'
)

col_filter = row_filters.ColumnQualifierRegexFilter('ambient_temp'.encode())

rows = table_weather.read_rows(row_set=row_set, filter_=col_filter)

for row in rows:
    print_row(row)

¿Cuáles fueron las diferentes medidas de temperatura ambiente para la planta 2 con fecha del 1 de junio desde las 6 de la mañana al mediodía?
Reading data for plant#4136001#2020-06-01 06:00:00:
Column Family telemetry
	ambient_temp: 23.470155033333327 @2020-06-01 06:00:00+00:00

Reading data for plant#4136001#2020-06-01 06:15:00:
Column Family telemetry
	ambient_temp: 23.67332510344828 @2020-06-01 06:15:00+00:00

Reading data for plant#4136001#2020-06-01 06:30:00:
Column Family telemetry
	ambient_temp: 23.979217633333327 @2020-06-01 06:30:00+00:00

Reading data for plant#4136001#2020-06-01 06:45:00:
Column Family telemetry
	ambient_temp: 24.209690400000003 @2020-06-01 06:45:00+00:00

Reading data for plant#4136001#2020-06-01 07:00:00:
Column Family telemetry
	ambient_temp: 24.567252344827587 @2020-06-01 07:00:00+00:00

Reading data for plant#4136001#2020-06-01 07:15:00:
Column Family telemetry
	ambient_temp: 24.835614033333336 @2020-06-01 07:15:00+00:00

Reading data for plant#4136001#

In [ ]:
rows = table_generation.read_rows()
for row in rows:
    print_row(row)

In [10]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")